### Conversational AI - Research

In [159]:
!pip install pgvector psycopg2 einops
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
62724.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
62731.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [182]:
import torch
import torch.nn.functional as F

import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from transitions import Machine
from dotenv import dotenv_values
from rich import print


secret = dotenv_values('../../.secret')

gpt3_5_turbo_recruiter  = dspy.OpenAI(
    model='gpt-3.5-turbo-0125',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

gpt4o  = dspy.OpenAI(
    model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)


llama3 = dspy.GROQ(
    model='llama3-70b-8192',
    api_key=secret['GROQ_API_KEY'],
    max_tokens=4096*2
)

dspy.settings.configure(lm=llama3)


def get_device():
    device = "cuda" \
        if torch.cuda.is_available() \
        else "mps" if torch.backends.mps.is_available() \
        else "cpu"
    if device == "mps":
        import os
        os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
        os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    return torch.device(device)


In [161]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel


tokenizer_embed = AutoTokenizer.from_pretrained('bert-base-uncased')
model_embed = AutoModel.from_pretrained('nomic-ai/nomic-embed-text-v1.5', trust_remote_code=True, safe_serialization=True)
model_embed.eval()


def embedd(text: str):
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    encoded_input = tokenizer_embed(text, padding=True, truncation=True, return_tensors='pt')

    # + matryoshka_dim = 512

    with torch.no_grad():
        model_output = model_embed(**encoded_input)

    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # + embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
    # + embeddings = embeddings[:, :matryoshka_dim]
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return np.array(embeddings)[0]



In [162]:
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

def persona_hub(persona_desc: str, subset: str) -> None:
    try:
        connection = psycopg2.connect(user="drfadul",
                                    password="dROG@dijoFadul",
                                    host="localhost",
                                    port="5432",
                                    database="synaia")
        
        register_vector(connection)
        cursor = connection.cursor()
        data = [
            (subset, persona_desc, embedd(text=persona_desc))
        ]

        execute_values(cursor, "INSERT INTO persona_hub (subset, persona_desc, persona_embedding) VALUES %s", data)
        connection.commit()


    except (Exception, psycopg2.Error) as error:
        print("Error while INSERTING data from PostgreSQL", error)

    finally:
        if connection:
            cursor.close()
            connection.close()



def retrieve_persona(text: str, K: int, negative_similarity: bool = False) -> list:
    personas = []
    try:
        connection = psycopg2.connect(user="drfadul",
                                    password="dROG@dijoFadul",
                                    host="localhost",
                                    port="5432",
                                    database="synaia")
        register_vector(connection)
        cursor = connection.cursor()
        text_embedd = embedd(text=text)
        data = (text_embedd, text_embedd, K)
    
        persona_hub_query = "SELECT subset, persona_desc, 1 - (persona_embedding <=> %s::vector) AS similarity FROM persona_hub ORDER BY persona_embedding <=> %s::vector LIMIT %s"
        if negative_similarity:
            persona_hub_query = "SELECT subset, persona_desc, 1 - (persona_embedding <=> %s::vector) AS similarity FROM persona_hub ORDER BY persona_embedding <=> %s::vector DESC LIMIT %s"

        cursor.execute(persona_hub_query, data)
        records = cursor.fetchall()

        for row in records:
            personas.append(
                (row[1], row[2])
            )
        
        return personas

    except (Exception, psycopg2.Error) as error:
        print("\x1b[1;31m Error while fetching data from PostgreSQL \x1b[1;31m", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()





In [163]:
from datasets import load_dataset
from tqdm import tqdm

sample_size = 200_000

# Load the dataset
persona_dataset = load_dataset("proj-persona/PersonaHub", data_files="persona.jsonl")['train']
persona_dataset = persona_dataset[:sample_size]
print(f"Total number of input personas: {len(persona_dataset['persona'])}")

Total number of input personas: 200000

In [164]:
if False:
    for persona in tqdm(persona_dataset['persona']):
        persona = persona.strip()
        persona_hub(persona, "persona")
    

In [165]:
import torch
import torch.nn.functional as F
from transformers import AlbertTokenizer, AlbertForSequenceClassification
import random

# Define the model name (the name should correspond to the fine-tuned model you want to use)
model_name = "JanSt/albert-base-v2_mbti-classification"
device = get_device()

# Load the tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertForSequenceClassification.from_pretrained(model_name)
model.eval()
model.to(device)


def get_personality_indicator(persona: str):
    # Tokenize the text
    inputs = tokenizer(persona, return_tensors='pt').to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)
    probabilities_list = probabilities.squeeze().tolist()
    probabilities_list = {model.config.id2label[i]: p for i, p in enumerate(probabilities_list)}
    p_type = max(probabilities_list, key=lambda x: probabilities_list[x])

    return p_type, probabilities_list



In [166]:
from datasets import load_dataset
from tqdm import tqdm


# Load the dataset
mb_dataset = load_dataset("kl08/myers-briggs-type-indicator")['train']
print(f"Total number of myers-briggs records: {len(mb_dataset)}")

Repo card metadata block was not found. Setting CardData to empty.


Total number of myers-briggs records: 8675

In [167]:
import random
from rich import print

rdm = random.randint(0, len(mb_dataset))
min_len_post = 150 # chars
min_posts = 10
N = len(mb_dataset)
post_list = []

for p_type, posts in zip(mb_dataset['type'][:N], mb_dataset['posts'][:N]):
    posts = posts.split("|||")
    posts = {post for post in posts if "http" not in post and "www" not in post and len(post) > min_len_post}
    if len(posts) > min_posts:
        post_list.append(
            {
                "type": p_type,
                "posts": posts
            }
        )

len(post_list)

In [192]:
K = 100
key_words = [
    ['Young', 'Modern', 'Joyful', 'Outspoken', 'Inexperienced', "Open-minded", 'Smart', "Change-resistant", 'selfish', 'Honest', 'Relaxed', "Job"]
]

__persona_applicant = []
for kw in [" ".join(k).strip() for k in key_words]:
    retrieved = retrieve_persona(text=kw, K=K)
    for r in retrieved:
        __persona_applicant.append(r[0])



key_words = [
    ["friendly", "knowledable", "professional", "women", "talkative", "Work", "Job"],
    # ["racing", "adventurer", "rich", "job"],
]

__persona_recruitment = []
for kw in [" ".join(k).strip() for k in key_words]:
    retrieved = retrieve_persona(text=kw, K=K)
    for r in retrieved:
        __persona_recruitment.append(r[0])



In [194]:
import random
import time
from typing import Optional
from IPython.display import display, HTML
from tqdm import tqdm


N_DATA = 100

RAW_FILE = "synth_raw_file_llama_round_2.txt"
JSON_FILE = "synth_conversations_llama_round_2.json"


def random_selection(elements: list, proba: float) -> Optional[str]:
    cond = [proba/len(elements) for _ in range(len(elements))]
    el = [None]
    el.extend(elements)
    weights = [1 - proba]
    weights.extend(cond)
    return random.choices(el, weights=weights, k=1)[0]



for N_position in tqdm(range(N_DATA), desc="SYNTH DATA $ 💉 COOKING ☕️", total=N_DATA, ncols=110):
    persona_applicant = random.choice(__persona_applicant)

    p_type_applicant, probabilities_list = get_personality_indicator(persona_applicant)

    persona_applicant_posts = random.choice([ p['posts'] for p in  post_list if p['type'] == p_type_applicant])

    persona_recruitment = random.choice(__persona_recruitment)

    p_type_recruitment, probabilities_list = get_personality_indicator(persona_recruitment)

    persona_recruitment_posts = random.choice([ p['posts'] for p in  post_list if p['type'] == p_type_recruitment])

    recruitment_posts = "\n".join(random.choices(list(persona_recruitment_posts), k=10))
    applicant_posts = "\n".join(random.choices(list(persona_applicant_posts), k=10))


    # display(HTML(f"<h3>Applicant: {persona_applicant}</h3>\n<h2>Personality: {p_type_applicant}</h3>"))
    # display(HTML(f"<h3>Recuiter: {persona_recruitment}</h3>\n<h2>Personality: {p_type_recruitment}</h3>"))


    prompt = f"""
    Generate a conversational chitchat dialogue between an applicant and a recruiter (Maria).
    All conversation is conducted through WhatsApp.
    The position to be filled is as an agent in a call center.

    Take into account the Personas and Myers–Briggs personality types indicator to enrich and make the dialogue more realistic:
    - Applicant Persona: {persona_applicant}
    - Applicant Indicator: {p_type_applicant}
    - Recruiter Persona: {persona_recruitment}
    - Recruiter Indicator: {p_type_recruitment}

    Follow the guidances:
    - The applicant initiates the conversation with a short greeting, it is possible that the applicant did not know the name of the recruiter.
    - The applicant uses informal language throughout the conversation and slangs.
    - The recruiter's goal is to be proactive and guide the applicant through a series of tasks.
    - The text includes metadata enclosed in pipe, You should NOT explicitly put this metadata in the generation.
    - Craft a realistic and engaging conversational dialogue that simulates a real-world recruitment chat.
    - Ensure the recruiter provides clear guidance and feedback between each task.
    - Aim to reflect the authentic dynamics of a recruitment process, highlighting both the recruiter's professional advice and the applicant's genuine responses.
    - No need to reiterate the information in the chat, as it's already captured in the submitted form. Let's move forward with the next steps!
    - There is no need to transcribe voice notes in chat.
    - Recruiter should be proactive.
    - Recruiter should push the applicant to complete each step.
    - In a new turn the applicant must explicitly indicate whether they completed a step or after sending a voice note.

    Tasks to be completed during the chat:
    1. Complete a basic form with required information, As a proactive recruiter, share the form.  |This step consists of the name, English level and accepting the terms and conditions. This form is shared within the chat|
    2. Answer a two-question assessment within a form to evaluate your skills, it is not necessary to write the questions in the chat as they are included in the form,  As a proactive recruiter, share the form. |The applicant must answer two questions in two long paragraphs|
    3. Record a voice note reading a provided text aloud and send it to the recruiter, As a proactive recruiter, share the text. |Write `PLACEHOLDER_1` intead of text, The text to be read is random and challenges the applicant's level of English.|
    4. Respond to an open-ended question with a voice note,  allowing the recruiter to assess their thoughts and opinion, as a proactive recruiter, share the open-ended question. |Write `PLACEHOLDER_2` intead of text.|
    """


    # gpt4o_output = gpt4o(prompt=prompt, temperature=0.7)
    # gpt4o_output = " ".join([out for out in gpt4o_output])

    llama3_output = llama3(prompt=prompt, temperature=0.3)
    llama3_output = " ".join([out for out in llama3_output])

    time.sleep(7)

    # print("----------------------------------------      SEAMLESS CONVERSATION      ------------------------------------------")
    # print(llama3_output)
    # print("--------------------------------------------------------------------------------------------------------------------\n\n\n")


    # proba 1.0
    system_task_completed = "Regardless of the conversation flow, for each completed task there is a system notification with a message like this: ***System***: Task N completed, for a scheduled task: ***System***: Scheduled by the user."
    # proba 0.7
    questions_and_concerns = "Between turns, incorporate the applicant's questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction, the Recruiter should assist the applicant in the new incorporated questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction."

    # proba 0.3
    busy_time_1 = "Restriction: NOT after last step have had completed, only one time between turns, the applicant expresses that they can't continue right now, are busy, or prefer to continue later. They might suggest any alternative way to stop the process and continue later. The Recruiter should assist the applicant by providing a form to schedule a time when the applicant can continue with the process. After the scheduling, simulate a time-elepsed and continue the conversation."
    busy_time_2 = "Restriction: NOT after last step have had completed, only one time between turns, the applicant expresses in an alternative way: 'It is very loud here, can we skip it for now?'.  The Recruiter should assist the applicant by providing a form to schedule a time when the applicant can continue with the process. After the scheduling, simulate a time-elepsed and continue the conversation."
    busy_time_3 = "Restriction: NOT after last step have had completed, only one time between turns, the applicant expresses in an alternative way: 'I am driving and I cant read. I will do it later.'.  The Recruiter should assist the applicant by providing a form to schedule a time when the applicant can continue with the process. After the scheduling, simulate a time-elepsed and continue the conversation."
    busy_time_4 = "Restriction: NOT after last step have had completed, only one time between turns, the applicant expresses in an alternative way: 'I prefer to continue in person. I am busy now.'.  The Recruiter should assist the applicant by saying that this process needs to be complete by this chat because is quickly and convenient for the applicant, also providing a form to schedule a time when the applicant can continue with the process. After the scheduling, simulate a time-elepsed and continue the conversation."
    busy_time_5 = "Restriction: NOT after last step have had completed, only one time between turns, the applicant expresses in an alternative way: 'I am sorry, I forgot. I'll get back to you when I'm ready to continue.'.  The Recruiter should assist the applicant by providing a form to schedule a time when the applicant can continue with the process. After the scheduling, simulate a time-elepsed and continue the conversation."

    # proba 0.3
    ask_for_missing_form = "Restriction: The Applicant cannot ask for a task that has already been completed, Sometimes the Applicant ask for resend of any specific form, task or question, reason for resend: the applicant may have missing it in the chat."

    # proba 0.2
    not_interested_1 = "In the middle of the dialogue, the applicant expresses that they don't want to continue or they are not interested anymore. The Recruiter should try to convince him or her that this is his or her best option. Also should assist the applicant by providing a feedback and sharing a form to schedule a time if the applicant change their mind to continue with the process. Only if the applicant does not want to continue and the process ends."
    not_for_me = "Only once, after completing a given task, the applicant expresses in an alternative way: 'I don't think this is for me.'. The Recruiter should try to convince him or her that this is his or her best option. Also should assist the applicant and motivate the applicant to continue by saying that this process is easy, quickly also convenient for the applicant."

    # proba 0.3
    not_interested_2_after_scheduling = "After the scheduling the process, the applicant expresses in an alternative way: 'I am sorry, I recently got a job. Thank you anyway.' in the form that they are not interested anymore. The Recruiter should assist the applicant by providing a feedback and sharing a form to schedule a time if the applicant change their mind to continue with the process. Only if the applicant does not want to continue and the process ends."
    not_interested_3_after_scheduling = "After the scheduling the process, the applicant expresses in an alternative way: 'thank you for following up, but I changed my mind.' in the form that they are not interested anymore. The Recruiter should try to convince him or her that this is his or her best option. Also should assist the applicant by providing a feedback and sharing a form to schedule a time if the applicant change their mind to continue with the process. Only if the applicant does not want to continue and the process ends."

    # proba 0.3
    ask_in_person = "In the middle of the dialogue, at a certain turn, the applicant asks if he or she can do the procedure personally. The Recruiter should assist the applicant by saying that this process needs to be complete by this chat because is quickly and convenient for the applicant."
    ask_for_different_way = "In the middle of the dialogue, at a certain turn, the applicant asks if he or she can do this in a different way because he or she is old school. The Recruiter should assist the applicant by saying that this process needs to be complete by this chat because is quickly and convenient for the applicant."
    ask_for_send_resume = "In the middle of the dialogue, at a certain turn, the applicant expresses, in an alternative way of say that if he or she can send his resume. The recruiter should assist the applicant by explaining that is convenient to complete the process within a chat  because is very effective and quicker and convenient for the applicant."
    do_over_the_phone = "Only once after Recruiter greetings, the applicant expresses, in an alternative way of say that their phone's screen is cracked and he or she can't read very well' with ther porpuse of do the process over the phone. The recruiter should assist the applicant by explaining that is convenient to complete the process within a chat  because is very effective and quicker and convenient for the applicant."

    # proba 0.3
    too_complicated = "Only once after Recruiter greetings, the applicant expresses in an alternative way: 'Can I just send you my resume? This is too complicated.'. The Recruiter should assist the applicant by saying that this process is easy and needs to be complete by this chat because is quickly and convenient for the applicant."
    how_long_is = "In the middle or at the end of the dialogue, the applicant expresses, in an alternative way, 'How long would it take? I have other job offers,' in the form of a clarifying question. The recruiter should assist the applicant by explaining that this process is more effective and quicker, taking days instead of weeks."

    # proba 0.4
    am_hired = "In the middle or at the end of the dialogue, the applicant expresses, in an alternative way, 'Am I hired?' in the form of receiving immediate feedback. The recruiter should assist the applicant by explaining that first need to be evaluate by a human recruiter and the process is very effective and quicker, taking days instead of weeks."
    immediate_feedback = "Only once, after completing a given task, does the applicant request immediate feedback. The recruiter should assist the applicant by explaining that this is being checked and needs to be evaluated by a human recruiter very efficiently and quickly."
    project_considered_for = "In the middle or at the end of the dialogue, the applicant expresses, in an alternative way, 'Which project am I being considered for?'. The recruiter should help the applicant by saying that there are many open positions, first need to be evaluate by a human recruiter and the process is very effective and quicker, taking days instead of weeks."
    remote_position = "Between turns, incorporate the applicant's questions, concerns, opinions, doubts, statements, and any other dialogue like if there are open remote position, this may naturally arise during the interaction, the Recruiter should assist the applicant in the new incorporated questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction."

    # proba 0.5
    ask_for_bonus = "In the middle or at the end of the dialogue, the applicant expresses, in an alternative way of say: 'Do you pay hiring bonus?'. The recruiter should help the applicant by saying yes, but the details must be in person with a human recruiter."
    salary_comission = "In the middle of the dialog, the applicant expresses, in an alternative way of say: 'What's the salary and commission like?'. The recruiter should help the applicant by saying that there are good salary and commission, but the details must be in person with a human recruiter."

    # proba 0.3
    work_here_in_past = "Only once after Recruiter greetings, the applicant expresses, in an alternative way of say: 'I worked there in the past, do I have to do this process?'. The recruiter should assist the applicant by explaining that this process is very effective and quicker, and asks to please complete it."
    i_have_good_english_but = "Only once, after completing a given task, the applicant expresses in an alternative way of say: 'If you see typos it's because I am not wearing my glasses. I have good english.' as an excuse. The recruiter should assist the applicant by explaining that this is being checked and needs to be evaluated by a human recruiter, dont worry this is efficiently and quickly."


    questions_and_concerns_ = random_selection([questions_and_concerns], 0.5)
    busy_time_ = random_selection([busy_time_1, busy_time_2, busy_time_3, busy_time_4, busy_time_5], 0.1)
    ask_for_missing_form_ = random_selection([ask_for_missing_form], 0.2)
    not_interested__ = random_selection([not_interested_1, not_for_me], 0.05)
    not_interested_after_scheduling_ = random_selection([not_interested_2_after_scheduling, not_interested_3_after_scheduling], 0.05)
    ask_in_person__ = random_selection([ask_in_person, ask_for_different_way, ask_for_send_resume, do_over_the_phone], 0.1)
    complicated__ = random_selection([too_complicated, how_long_is], 0.1)
    feedback__ = random_selection([am_hired,  immediate_feedback, project_considered_for, remote_position], 0.2)
    salary_and_bonus_ = random_selection([ask_for_bonus, salary_comission], 0.2)
    another_ = random_selection([work_here_in_past, i_have_good_english_but], 0.05)

    concerns = [questions_and_concerns_, busy_time_, ask_for_missing_form_, not_interested__, not_interested_after_scheduling_, ask_in_person__, complicated__, feedback__, salary_and_bonus_, another_]
    random.shuffle(concerns)


    new_prompt = """Given the following WhatsApp Chat Conversation enclosed in a triple backtick, to simulates a real-world recruitment chat perform the following:\n"""

    #llama3
    new_prompt += """MODIFY THE GIVEN CONVERSATION INSERTING BETWEEN TURNS SOME DIALOGS ACCORDING TO THE FOLLOWING RULES:\n"""
    new_prompt += """- INSERT AFTER A COMPLETED TASK A MESSAGE LIKE THIS: ***SYSTEM***: TASK N COMPLETED, FOR A SCHEDULED TASK: ***SYSTEM***: SCHEDULED BY THE USER.\n"""

    for concern in concerns:
        if concern:
            new_prompt += f"""- {concern}\n"""

    new_prompt += f"""- {system_task_completed}\n\n"""
    new_prompt += f"""```{llama3_output}```"""

    

    # print("------------------------------------------     PROMPT GENERATED       --------------------------------------------")
    # print(new_prompt)
    # print("--------------------------------------------------------------------------------------------------------------------\n\n\n")


    new_prompt_ouput = llama3(prompt=new_prompt, temperature=0.2)
    new_prompt_ouput = " ".join([out for out in new_prompt_ouput])

    # print("------------------------------------------    MODIFIED CONVERSATION    --------------------------------------------")
    # print(new_prompt_ouput)
    # print("--------------------------------------------------------------------------------------------------------------------\n\n\n")
    
    
    new_prompt_ouput += "\n\n\n"
    with open(RAW_FILE, "a") as raw:
        raw.write(new_prompt_ouput)

    
    time.sleep(7)


    format_instruct = '''GIVEN THE FOLLOWING WHATSAPP CHAT ENCLOSED IN TRIPLE BACKTICK CONVERSATION FORMAT AS JSON AS FOLLOW AND RETURN ONLY THE JSON, DO NOT INCLUDE EXTRA INFORMATION LIKE 'Here is the JSON representation of the ...':
    {
       "conversation_''' + str(N_position) + '''": [
            {
            "speaker": "Applicant",
            "message": "String ... "
            },
            {
            "speaker": "Recruiter",
            "message": "String ..."
            },
            {
            "speaker": "Applicant",
            "message": "String ..."
            },
            {
            "speaker": "Recruiter",
            "message": "String ..."
            },
            {
            "speaker": "Applicant",
            "message": "String ..."
            },
            {
            "speaker": "System",
            "message": "String ..."
            },
       ]
    },
    '''

    format_instruct += f"\n\n ```{new_prompt_ouput}```"

    format_output = llama3(prompt=format_instruct, temperature=0.0)
    format_output = " ".join([out for out in format_output])

    with open(JSON_FILE, "a") as raw:
        raw.write(format_output)


    time.sleep(7)





SYNTH DATA $ 💉 COOKING ☕️:   0%|                                                    | 0/100 [00:00<?, ?it/s]

SYNTH DATA $ 💉 COOKING ☕️: 100%|██████████████████████████████████████████| 100/100 [59:57<00:00, 35.97s/it]


img hidde here
<!-- <img src="https://upload.wikimedia.org/wikipedia/commons/1/1f/MyersBriggsTypes.png" alt="drawing" width="700"/> -->

### Extracting TAGS

In [230]:
llama3 = dspy.GROQ(
    model='llama3-70b-8192',
    api_key=secret['GROQ_API_KEY'],
    max_tokens=4096*2,
)




In [231]:
import json 
import random
import time

with open('synth_data/synth_conversations_llama_round_2.json', "r") as synth:
    conversations = json.load(synth)



TAGS = """# Task Description

Your task is to condense a message from a recruitment service dialog into a concise word. 

To achieve this, follow these guidelines:

1. **Brevity is key**: Limit your summary to 3 words or less, focusing on the essential information.
2. **Event-centric**: Extract the main event, issue, query, solution, or feedback from the message.



# Example

Input: "Oh, I forgot to ask, could you resend the assessment form link? I might have missed something."

Output: "#resend #assessment"
"""


with_tags = []
for i in tqdm(range(len(conversations)), desc="GENERATING TAGS ☕️", total=len(conversations), ncols=110):
  conversation = conversations[i]
  for k, value in conversation.items():
    current = conversation[k]
    # TAGS = TAGS.format("\n".join([f"{d['speaker']}: {d['message']}" for d in current]))
    for j, v in enumerate(value):
      message = v['message']
      TAGS = f"""
GIVEN THE FOLLOWING DIALOGUE TURN ENCLOSED IN TRIPLE BACKTICKS:
```{message}```


PERFORM THE FOLLOWING TASKS:

Condense the message into a concise hashtag, such as "#interest" or "#position".
Extract the main event, issue, query, solution, or feedback from the message.
THE OUTPUT SHOULD BE 1 TO 3 WORDS, E.G., "#introduction" OR "#callcenter".
DO NOT INCLUDE ANY ADDITIONAL TEXT BEYOND THE FINAL HASHTAGS.


TAKE INTO ACCOUNT THE COMPLETE CONVERSATION AS A CONTEXT:
```{" ".join([f"{d['speaker']}: {d['message']}" for d in current])}```
            """
      
      tags_output = llama3(prompt=TAGS, temperature=0.2)
      tags_output = " ".join([out for out in tags_output])
      current[j]['tags'] = tags_output

      time.sleep(random.randint(1, 3))

    with open("synth_conversations_llama_round_2_with_tags.json", "w") as raw:
      with_tags.append(current)
      json.dump(with_tags, raw, indent=4)


# print(conversations[0])


GENERATING TAGS ☕️:  16%|███████▋                                        | 16/100 [37:00<3:07:08, 133.67s/it]INFO:backoff:Backing off request(...) for 0.1s (groq.InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}})


Backing off 0.1 seconds after 1 tries calling function <function GroqLM.request at 0x1747d9f30> with kwargs {'temperature': 0.2}


INFO:backoff:Backing off request(...) for 1.8s (groq.InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}})


Backing off 1.8 seconds after 2 tries calling function <function GroqLM.request at 0x1747d9f30> with kwargs {'temperature': 0.2}


INFO:backoff:Backing off request(...) for 0.3s (groq.InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}})


...

INFO:backoff:Backing off request(...) for 2.9s (groq.InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}})


Backing off 2.9 seconds after 4 tries calling function <function GroqLM.request at 0x1747d9f30> with kwargs {'temperature': 0.2}


INFO:backoff:Backing off request(...) for 4.4s (groq.InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}})


Backing off 4.4 seconds after 5 tries calling function <function GroqLM.request at 0x1747d9f30> with kwargs {'temperature': 0.2}


GENERATING TAGS ☕️: 100%|███████████████████████████████████████████████| 100/100 [3:57:24<00:00, 142.44s/it]
